# Colorization of Densified Point Clouds

The densified point cloud ignores all color in the original point cloud. This script uses the original point cloud to colorize the densified point cloud based on a weighted nearest neighbor search.

In [ ]:
# Original point cloud is given in .npy format, with XYZRGBL data
# Densified point cloud is given in .xyz format, with XYZ data (can be read as plain text)

import numpy as np
import glob
from sklearn.neighbors import KDTree

original_point_cloud_paths = glob.glob("data/point_clouds/*.npy")
densified_point_cloud_paths = glob.glob("data/point_clouds/*.xyz")

# If a point in the densified point cloud is the same as a point in the original point cloud, it is given the color of the original point cloud
# we will need to take that into account when weighting the nearest neighbors

for i in range(len(original_point_cloud_paths)):
    original_point_cloud = np.load(original_point_cloud_paths[i])
    densified_point_cloud = np.loadtxt(densified_point_cloud_paths[i])
    # add columns for the color of the densified point cloud and last column for the label
    densified_point_cloud = np.hstack((densified_point_cloud, np.zeros((len(densified_point_cloud), 4))))

    # We will use a KDTree to find the nearest neighbors
    tree = KDTree(original_point_cloud[:, :3], leaf_size=2)

    # Find the nearest neighbors of each point in the densified point cloud
    distances, indices = tree.query(densified_point_cloud[:, :3], k=3)
    
    # We will weight the nearest neighbors by the inverse of their distance
    weights = 1 / distances
    
    # Normalize the weights
    weights = weights / np.sum(weights, axis=1)[:, None]
    
    # Colorize the densified point cloud
    for j in range(len(densified_point_cloud)):
        densified_point_cloud[j, 3:] = np.sum(weights[j] * original_point_cloud[indices[j], 3:], axis=0)
        
        # the last column should be set to 5 (this is "building", and all we are densifying are buildings)
        densified_point_cloud[j, -1] = 5
    
    # Save the colored point cloud
    np.savetxt("data/point_clouds/densified_point_cloud_{}.xyzrgbl".format(i), densified_point_cloud, fmt="%.6f %.6f %.6f %d %d %d %d %d")
    # this is a weird format so we may want to save as .las instead (or .ply)
    
